# Altair Choropleths play

Here I load in James' presidential polls dataset from the teaching notebook in this dir and attempt some rudimentary WAYS ideas.

In [1]:
%%capture
# capture suppresses output of the below:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -r requirements.txt

In [2]:
import geopandas as gpd 
import pandas as pd
import altair as alt

geo_states = gpd.read_file('choropleth_teaching/gz_2010_us_040_00_500k.json')
df_polls = pd.read_csv('choropleth_teaching/presidential_poll_averages_2020.csv')

In [3]:
df_polls.head()

,cycle,state,modeldate,candidate_name,pct_estimate,pct_trend_adjusted
0,2020,Wyoming,11/03/2020,Joseph R. Biden Jr.,30.81486,30.82599
1,2020,Wisconsin,11/03/2020,Joseph R. Biden Jr.,52.12642,52.09584
2,2020,West Virginia,11/03/2020,Joseph R. Biden Jr.,33.49125,33.51517
3,2020,Washington,11/03/2020,Joseph R. Biden Jr.,59.34201,59.39408
4,2020,Virginia,11/03/2020,Joseph R. Biden Jr.,53.74120,53.72101


On how many dates was the polling done?

In [4]:
len(df_polls.modeldate.unique())

251

Filter our poll data to remove third party candidates.

In [5]:
df_polls = df_polls[
    (df_polls.candidate_name == 'Donald Trump') |
    (df_polls.candidate_name == 'Joseph R. Biden Jr.')
]

The geo_states variable has polygons for each state.

In [6]:
geo_states.head()

,GEO_ID,STATE,NAME,LSAD,CENSUSAREA,geometry
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ..."
1,0400000US25,25,Massachusetts,,7800.058,"MULTIPOLYGON (((-70.83204 41.60650, -70.82373 ..."
2,0400000US26,26,Michigan,,56538.901,"MULTIPOLYGON (((-88.68443 48.11579, -88.67563 ..."
3,0400000US30,30,Montana,,145545.801,"POLYGON ((-104.05770 44.99743, -104.25015 44.9..."
4,0400000US32,32,Nevada,,109781.180,"POLYGON ((-114.05060 37.00040, -114.04999 36.9..."


In [7]:
trump_data = df_polls[
    df_polls.candidate_name == 'Donald Trump'
]

biden_data = df_polls[
    df_polls.candidate_name == 'Joseph R. Biden Jr.'
]

Our spatial and poll data have the name of the state in common. We will change the name of the state to NAME to match our geospatial dataframe.

In [8]:
trump_data.columns = ['cycle', 'NAME', 'modeldate', 'candidate_name', 'pct_estimate', 'pct_trend_adjusted']
biden_data.columns = ['cycle', 'NAME', 'modeldate', 'candidate_name', 'pct_estimate', 'pct_trend_adjusted']

We can join the geospatial and poll data using the NAME column (the name of the state).

In [9]:
# Create seperate date frame for trump and biden
# Add the poll data
geo_states_trump = geo_states.merge(trump_data, on='NAME')
geo_states_biden = geo_states.merge(biden_data, on='NAME')

In [10]:
(geo_states_trump.modeldate == '11/03/2020').value_counts()

False    9972
True       51
Name: modeldate, dtype: int64

In [11]:
geo_states_biden.head()

,GEO_ID,STATE,NAME,LSAD,CENSUSAREA,geometry,cycle,modeldate,candidate_name,pct_estimate,pct_trend_adjusted
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,11/03/2020,Joseph R. Biden Jr.,53.31518,53.32106
1,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,11/02/2020,Joseph R. Biden Jr.,53.31518,53.32106
2,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,11/01/2020,Joseph R. Biden Jr.,53.70696,53.91672
3,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,10/31/2020,Joseph R. Biden Jr.,53.30098,53.55544
4,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,10/30/2020,Joseph R. Biden Jr.,53.34166,53.64551


### Create choropleth plot function

In [12]:
def usa_choro(candidate_geo_states, color, title):
    """Plot a choropleth of the US states with the candidate vote percentage mapped to the color;
    pass in altair bin and scale objects"""
    chart = alt.Chart(candidate_geo_states, title=title).mark_geoshape()
    chart = chart.encode(
        color,
        tooltip=['NAME', 'pct_estimate']
    ).properties(
        width=500,
        height=300
    ).project(
        type='albersUsa'
    )
    return chart

### Import WAYS "metavisualisation" histogram

In [13]:
import sys; sys.path.insert(0, '..')
from ways_py.ways import Ways

In [14]:
Ways.altair_meta_hist

<function ways_py.ways.Ways.altair_meta_hist(dataframe: pandas.core.frame.DataFrame, column: str, bin: altair.vegalite.v4.schema.core.BinParams, color: altair.vegalite.v4.schema.channels.Color) -> altair.vegalite.v4.api.Chart>

## Adding interactivity with Jupyter interact widgets

Here I attempt to offer the user working on the the US presidential poll choropleth visualisation some options related to the colour-binning.

In [39]:
from ipywidgets import interact, widgets, HBox, VBox, GridspecLayout

In [16]:
# Get an ordered list of the dates (as strings) on which polling occured
import datetime
unsorted_datestrings = list(set(list(geo_states_biden['modeldate'])))
dates = sorted(unsorted_datestrings, key=lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))

Get a list of possible altair scales from [here](https://altair-viz.github.io/user_guide/generated/core/altair.ScaleType.html#altair.ScaleType) and create a list:

In [17]:
scales = ['linear', 'log', 'pow', 'sqrt', 'symlog', 'identity', 'sequential', 'time', 'utc', 'quantile', 'quantize', 'threshold', 'bin-ordinal', 'ordinal', 'point', 'band']

In [ ]:
color_picker = widgets.ColorPicker(
    concise=True,
    description='Background color:',
    value='#efefef',
)
color_picker

In [75]:
items = [widgets.Label(str(i)) for i in range(4)]
items

[Label(value='0'), Label(value='1'), Label(value='2'), Label(value='3')]

In [83]:
# Create and arrange widgets
grid_matrix = []
candidate = widgets.Dropdown(options=['Trump', 'Biden'], description = 'Candidate')
grid_matrix.append(candidate)
date = widgets.SelectionSlider(value='11/03/2020', options=dates, description='Date')
grid_matrix.append(date)
bin = widgets.Checkbox(value=True, description='Bin')
grid_matrix.append(bin)
maxbins = widgets.IntSlider(value=100, min=2, max=100, step=1, description='Max Bins', continuous_update=False)
grid_matrix.append(maxbins)
scale = widgets.Dropdown(value='linear', options=scales, description = 'Scales')
grid_matrix.append(scale)
extent = widgets.IntRangeSlider(value=[0,100], min=0, max=100, description='Extent')
grid_matrix.append(extent)
colorschemetype = widgets.RadioButtons(options=['Scheme', 'Range'], description='Color Scheme Type')
grid_matrix.append(colorschemetype) #stacked?
colorscheme = widgets.Dropdown(options=['redblue', 'darkred'], description = 'Color Scheme')
grid_matrix.append(colorscheme)
colorrange = widgets.Dropdown(options=['redpurpleblue', 'redorangeyellow'], description = 'Color Range')
grid_matrix.append(colorrange)
# grid = widgets.GridspecLayout(grid_matrix)
grid = widgets.GridBox(grid_matrix, layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))

# Define interactive plot function
# @interact
def interact_usa_choro(candidate,
                       date,
                       bin,
                       maxbins,
                       scale,
                       extent,
                       colorschemetype,
                       colorscheme,
                       colorrange
                      ):
    
#     display(grid)
    
    # Chooses the candidate dataset
    candidate_geo_states = geo_states_biden
    if candidate == 'Trump':
        candidate_geo_states = geo_states_trump

    # Choose which polling date to display
    candidate_geo_states = candidate_geo_states[
        (candidate_geo_states.modeldate == date)
    ]

    # Give the choropleth plot a title
    title = 'Poll estimate for ' + candidate + ' on ' + date

    # Creates the bin, scale and color objects, to be passed to both plots
    if bin: # if bin is False, leave as bool
        bin = alt.Bin(maxbins=maxbins, extent=extent)
    if colorschemetype == 'Scheme':
        scale = alt.Scale(type=scale, scheme=colorscheme)
    elif colorschemetype == 'Range':
        if colorrange == 'redpurpleblue':
            scale = alt.Scale(type=scale, range=['red', 'purple', 'blue'])
        elif colorrange == 'redorangeyellow':
            scale = alt.Scale(type=scale, range=['red', 'orange', 'yellow'])
    column = "pct_estimate"
    color = alt.Color(column,
                      legend=None,
                      bin=bin,
                      scale=scale
                     )
    display(Ways.altair_meta_hist(candidate_geo_states, column, bin, color) | usa_choro(candidate_geo_states, color, title))

out = widgets.interactive_output(
    interact_usa_choro,
    {
        'candidate': candidate,
        'date': date,
        'bin': bin,
        'maxbins': maxbins,
        'scale': scale,
        'extent': extent,
        'colorschemetype': colorschemetype,
        'colorscheme': colorscheme,
        'colorrange': colorrange
    }
)
display(grid, out)





GridBox(children=(Dropdown(description='Candidate', options=('Trump', 'Biden'), value='Trump'), SelectionSlide…

Output()

Notes:
1. Setting the scale to `log` doesn't work unless `bin` is **unchecked**
2. Because the `extent` data range plugs into the `alt.Bin` object, this will only work if `bin` is **checked**
3. There is no way to pre-populate dropdowns with all available options for a given function argument